<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/DRLforMultipleStocksTradingUsingFinRL/blob/main/FractionsTest3/3_21_fraksi5_TugasAkhirS1_Shinta_DRLforMultipleStocksTradingUsingFinRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook ini dijalankan menggunakan email shintaroudlotulhanafia@gmail.com

# Deep Reinforcement Learning untuk Jual-Beli Saham (Dari Awal): Jual-beli Banyak Saham 

* **Pytorch Version** 



# Daftar Isi:

* [1. Pendeskrisian Tugas](#0)
* [2. Instalasi Paket Python](#1)
    * [2.1. Instalasi Paket](#1.1)    
    * [2.2. Daftar Paket Python](#1.2)
    * [2.3. Impot Paket](#1.3)
    * [2.4. Membuat Folder](#1.4)
* [3. Mengunduh Data](#2)
* [4. Memproses Data](#3)        
    * [4.1. IndiKator Teknikal](#3.1)
    * [4.2. Melakukan *Feature Engineering*](#3.2)
* [5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style](#4)  
    * [5.1. Pemisahan Data](#4.1)  
    * [5.2. Lingkungan untuk Pelatihan *(Training)*](#4.2)    
* [6. Melatih Agen DRL](#5)
    * [6.1. Agen 1: A2C](#5.1)
    * [6.2. Agen 2: PPO](#5.2)
    * [6.3. Agen 3: TD3](#5.3)
    * [6.4. Agen 4: SAC](#5.4)
    * [6.5. Agen 5: DDPG](#5.5)
* [7. Melakukan Jual-Beli](#6)
    * [7.1. Performa di Dalam Sampel](#6.1)
    * [7.2. Performa di Luar Sampel](#6.2)
    * [7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi](#6.3)
      * [7.3.1. Agen 1: A2C](#6.3.1)
      * [7.3.2. Agen 2: PPO](#6.3.2)
      * [7.3.3. Agen 3: TD3](#6.3.3)
      * [7.3.4. Agen 4: SAC](#6.3.4)
      * [7.3.5. Agen 5: DDPG](#6.3.5)
* [8. Performa Backtesting](#7)  
    * [8.1. Status BackTesting](#7.1)
    * [8.2. Gambaran BackTesting](#7.2)

<a id='0'></a>
# Bagian 1. Pendeskrisian Tugas

Agen DRL dilatih untuk melakukan jual-beli saham. Tugas tersebut dimodelkan sebagai Markov Decision Process (MDP), dengan fungsi dan tujuannya adalah memaksimalkan pengembalian pengembalian kumulatif *(cummulative return)* yang diharapkan.

Definisi state-action-reward pada algoritam DRL kasus kali ini adalah sebagai berikut:

* **State s**: Kondisi atau *state* mewakili persepsi agen tentang lingkungan pasar. Sama seperti *trader* manusia yang menganalisis berbagai informasi, agen juga secara pasif mengamati banyak fitur dan belajar dengan cara berinteraksi dengan lingkungan pasar (biasanya dengan menjalankan ulang data historis).

* **Tindakan atau *action* a**: Ruang aksi mencakup aksi atau tindakan yang dapat dilakukan agen di setiap status atau kondisi. Misalnya, a {−1, 0, 1}, -1 berarti menjual, 0 berarti menahan, dan 1 berarti membeli. Ketika suatu aksi mengoperasikan beberapa saham, maka, a {−k, ..., 1, 0, 1, ..., k}. Misalnya, "Beli
10 saham TLKM" atau "Jual 10 saham TLKM" maka masing-masing nilai a adalah 10 atau -10.

* **Fungsi *reward* atau imbalan r(s, a, s′)**: *Reward* adalah insentif bagi agen untuk mempelajari kebijakan yang lebih baik. Misalnya *reward* dapat berupa perubahan nilai portofolio saat mengambil a pada keadaan s dan tiba pada keadaan baru s', yaitu, r(s, a, s′) = v′-v, v′ mewakili nilai portofolio pada keadaan s′ dan v mewakili nilai portofolio pada keadaan s.

* **Environment atau lingkungan jual-beli**: saham penyusun indeks JII dengan tanggal sesuai periode pengujian yang diatur.


Data untuk studi kasus ini diperoleh dari Yahoo Finance API. Data berisi harga *Open-High-Low-Close* dan *Volume*.

<a id='1'></a>
# Bagian 2. Instalasi Paket Python

<a id='1.1'></a>
## 2.1. Instalasi Paket


In [3]:
# install finrl library
!pip install git+https://github.com/shintaroudlotulhanafia/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shintaroudlotulhanafia/FinRL.git to /tmp/pip-req-build-wp4x543a
  Running command git clone -q https://github.com/shintaroudlotulhanafia/FinRL.git /tmp/pip-req-build-wp4x543a
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-hiq4ebck/pyfolio_daca545e9b6e423493ea01f238075ffc
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-hiq4ebck/pyfolio_daca545e9b6e423493ea01f238075ffc
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-hiq4ebck/elegantrl_ceedd2bd5ff44db8ad3f63e68f05dab4
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-hiq4ebck/elegantrl_ceedd2bd5ff44db8ad3f63e68f05dab4
     |████████████████████████████████| 2.3 MB 14.4 MB/s 
     |████████████████████████████████| 234 kB 78.0 MB/s 
     |█████████████████████

<a id='1.2'></a>
## 2.2. Daftar Paket Python
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Impot Paket

* Modul Python **pandas** digunakan untuk menganalisis dan memanipulasi data.
* Modul Python **numpy** digunakan untuk memproses larik atau array.
* Modul Python **matplotlib** digunakan membuat visualisasi data dalam dua dimensi.
* Modul Python **matplotlib.pyplot** adalah kumpulan fungsi yang membuat matplotlib berfungsi seperti MATLAB.
* Modul Python **Datetime** menyediakan sejumlah fungsi untuk menangani tanggal, waktu, dan interval waktu. Date dan datetime adalah objek dalam Python, bukan string atau timestamps.
* **YahooDownloader** menyediakan metode untuk mengambil data saham harian dari API Keuangan Yahoo!
* **FeatureEngineer** menyediakan metode untuk preprocessing data harga saham
* **data_split** membagi dataset menjadi data pelatihan dan data pengujian berdasarkan tanggal
* **StockTradingEnv** Lingkungan perdagangan saham untuk OpenAI gym
* **DRLAgent** menyediakan implementasi untuk algoritma DRL
* **DataProcessor** memproses data menggunakan prosesor data terpadu
* **backtest_stats** menghitung statistik *backtesting*
* **backtest_plot** membuat dan menampilkan plot ringkasan laporan *backtesting*.
* **get_daily_return** 
* **get_baseline** mengunduh data berdasarkan suatu indeks pada periode waktu tertentu.
* Metode **sys.path.append()** digunakan untuk menambahkan jalur sementara. Dengan demikian, jalur tersebut akan valid untuk sebuah sesi, misalnya.
* Python **itertool** adalah modul yang menyediakan berbagai fungsi yang bekerja pada iterator untuk menghasilkan iterator yang kompleks. Modul ini berfungsi sebagai alat yang cepat dan hemat memori.

In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


<a id='1.4'></a>
## 2.4. Membuat Folder

In [5]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Mengunduh Data

Yahoo Finance menyediakan data saham, berita keuangan, laporan keuangan, dll, secara gratis.
* FinRL *Library* menggunakan kelas **YahooDownloader** di FinRL-Meta untuk mengambil data melalui Yahoo Finance API
* Batas Pemanggilan: Menggunakan API Publik (tanpa autentikasi), Pengguna dibatasi hingga 2.000 permintaan per jam per IP (atau hingga total 48.000 permintaan per hari).

In [6]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2017-12-31'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2019-12-20'

In [7]:
#Data diperoleh dari idx.co.id diakses pada tanggal 27/08/2022
#Fraksi 1: x<Rp200,00
fraksi1 = ['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

#Fraksi 2: Rp200,00<= x <Rp500,00
fraksi2 = ['BMTR.JK','BTON.JK','FORU.JK','GEMA.JK']

#Fraksi 3: Rp500,00<= x <Rp2.000,00
fraksi3 = ['AKRA.JK','BRPT.JK','KLBF.JK','MEDC.JK']

#Fraksi 4: Rp2000,00<= x <Rp5000,00
fraksi4 = ['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

#Fraksi 5: >=Rp5000,00
fraksi5 =['INCO.JK','INDF.JK','INTP.JK','UNTR.JK']

In [161]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = fraksi5).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10912, 8)


In [162]:
df.nunique()

date       2728
open       1787
high       1851
low        1822
close      4891
volume    10004
tic           4
day           5
dtype: int64

In [163]:
print(df['tic'].unique())

['INCO.JK' 'INDF.JK' 'INTP.JK' 'UNTR.JK']


* **sort_values()** berfungsi mengurutkan bingkai data dalam urutan Ascending atau Descending dari kolom yang dilewati.
* **ignore_index** berfungsi untuk menentukan untuk mereset indeks mulai dari nol. Secara default disetel *false*.
* **head()** digunakan untuk mendapatkan n baris pertama.

In [164]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,2050.000000,2300.000000,2030.000000,1792.011108,30353500,INCO.JK,0
1,2009-01-05,980.000000,980.000000,920.000000,638.533020,154640500,INDF.JK,0
2,2009-01-05,4600.000000,5050.000000,4600.000000,3106.131348,3534000,INTP.JK,0
3,2009-01-05,4326.423828,5071.530273,4326.423828,3122.124756,25673281,UNTR.JK,0
4,2009-01-06,2300.000000,2325.000000,2100.000000,1733.576172,21013000,INCO.JK,1


<a id='3'></a>
# Bagian 4: Memproses Data

Periksa data yang hilang dan melakukan *feature engineering* untuk mengubah data menjadi *state*.
* **Menambahkan indikator teknis**

  Dalam praktik jual-beli, berbagai informasi perlu diperhitungkan, seperti harga historis, kepemilikan saham saat ini, indikator teknis, dll. Indikator teknis yang dapat digunakan antara lain, MACD, RSI, CCI, ADX, Bollinger Bands, dll. 
* **Menambahkan indeks turbulensi**

  Risk-aversion mencerminkan seberapa berani investor melakukan jual-beli saham dengan risiko yang tinggi. Hal ini mempengaruhi strategi trading investor ketika menghadapi tingkat volatilitas pasar yang berbeda. Untuk mengendalikan risiko dalam skenario terburuk, seperti krisis keuangan tahun 1998, FinRL menggunakan indeks turbulensi yang mengukur fluktuasi harga aset yang ekstrem.

In [165]:
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30"
]

Membersihkan data mentah dari nilai-nilai yang hilang. Sehingga saham dengan data yang hilang dihapus dari daftar.

In [166]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2759, 8)
Successfully added vix
Successfully added turbulence index


In [167]:
processed.nunique()

date           2651
open           1775
high           1835
low            1809
close          4831
volume         9764
tic               4
day               5
macd          10601
rsi_30         9615
cci_30        10599
vix            1441
turbulence     2397
dtype: int64

In [168]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['INCO.JK', 'INDF.JK', 'INTP.JK', 'UNTR.JK']

In [169]:
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
list_date

['2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-

In [170]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-05', 'INCO.JK'),
 ('2009-01-05', 'INDF.JK'),
 ('2009-01-05', 'INTP.JK'),
 ('2009-01-05', 'UNTR.JK'),
 ('2009-01-06', 'INCO.JK'),
 ('2009-01-06', 'INDF.JK'),
 ('2009-01-06', 'INTP.JK'),
 ('2009-01-06', 'UNTR.JK'),
 ('2009-01-07', 'INCO.JK'),
 ('2009-01-07', 'INDF.JK'),
 ('2009-01-07', 'INTP.JK'),
 ('2009-01-07', 'UNTR.JK'),
 ('2009-01-08', 'INCO.JK'),
 ('2009-01-08', 'INDF.JK'),
 ('2009-01-08', 'INTP.JK'),
 ('2009-01-08', 'UNTR.JK'),
 ('2009-01-09', 'INCO.JK'),
 ('2009-01-09', 'INDF.JK'),
 ('2009-01-09', 'INTP.JK'),
 ('2009-01-09', 'UNTR.JK'),
 ('2009-01-10', 'INCO.JK'),
 ('2009-01-10', 'INDF.JK'),
 ('2009-01-10', 'INTP.JK'),
 ('2009-01-10', 'UNTR.JK'),
 ('2009-01-11', 'INCO.JK'),
 ('2009-01-11', 'INDF.JK'),
 ('2009-01-11', 'INTP.JK'),
 ('2009-01-11', 'UNTR.JK'),
 ('2009-01-12', 'INCO.JK'),
 ('2009-01-12', 'INDF.JK'),
 ('2009-01-12', 'INTP.JK'),
 ('2009-01-12', 'UNTR.JK'),
 ('2009-01-13', 'INCO.JK'),
 ('2009-01-13', 'INDF.JK'),
 ('2009-01-13', 'INTP.JK'),
 ('2009-01-13', 'UNT

In [171]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2050.000000,2300.000000,2030.000000,1792.011108,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,INDF.JK,980.000000,980.000000,920.000000,638.533020,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,INTP.JK,4600.000000,5050.000000,4600.000000,3106.131348,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,UNTR.JK,4326.423828,5071.530273,4326.423828,3122.124756,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,INCO.JK,2300.000000,2325.000000,2100.000000,1733.576172,21013000.0,1.0,-1.311040,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNTR.JK,22000.000000,22375.000000,21750.000000,19864.078125,4607100.0,1.0,-78.720781,52.760272,20.354918,12.290000,0.951751
15996,2019-12-18,INCO.JK,3600.000000,3600.000000,3440.000000,3418.002930,14110300.0,2.0,27.673484,51.602337,90.205381,12.580000,4.030024
15997,2019-12-18,INDF.JK,7975.000000,7975.000000,7900.000000,7000.262207,8303900.0,2.0,11.639505,53.440010,59.891878,12.580000,4.030024
15998,2019-12-18,INTP.JK,20000.000000,20075.000000,19800.000000,17098.693359,2290000.0,2.0,-13.626100,49.586669,-17.159829,12.580000,4.030024


In [172]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2050.000000,2300.000000,2030.000000,1792.011108,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,INDF.JK,980.000000,980.000000,920.000000,638.533020,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,INTP.JK,4600.000000,5050.000000,4600.000000,3106.131348,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,UNTR.JK,4326.423828,5071.530273,4326.423828,3122.124756,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,INCO.JK,2300.000000,2325.000000,2100.000000,1733.576172,21013000.0,1.0,-1.311040,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNTR.JK,22000.000000,22375.000000,21750.000000,19864.078125,4607100.0,1.0,-78.720781,52.760272,20.354918,12.290000,0.951751
15996,2019-12-18,INCO.JK,3600.000000,3600.000000,3440.000000,3418.002930,14110300.0,2.0,27.673484,51.602337,90.205381,12.580000,4.030024
15997,2019-12-18,INDF.JK,7975.000000,7975.000000,7900.000000,7000.262207,8303900.0,2.0,11.639505,53.440010,59.891878,12.580000,4.030024
15998,2019-12-18,INTP.JK,20000.000000,20075.000000,19800.000000,17098.693359,2290000.0,2.0,-13.626100,49.586669,-17.159829,12.580000,4.030024


In [173]:
processed_full = processed_full.sort_values(['date','tic'])
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2050.000000,2300.000000,2030.000000,1792.011108,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,INDF.JK,980.000000,980.000000,920.000000,638.533020,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,INTP.JK,4600.000000,5050.000000,4600.000000,3106.131348,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,UNTR.JK,4326.423828,5071.530273,4326.423828,3122.124756,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,INCO.JK,2300.000000,2325.000000,2100.000000,1733.576172,21013000.0,1.0,-1.311040,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNTR.JK,22000.000000,22375.000000,21750.000000,19864.078125,4607100.0,1.0,-78.720781,52.760272,20.354918,12.290000,0.951751
15996,2019-12-18,INCO.JK,3600.000000,3600.000000,3440.000000,3418.002930,14110300.0,2.0,27.673484,51.602337,90.205381,12.580000,4.030024
15997,2019-12-18,INDF.JK,7975.000000,7975.000000,7900.000000,7000.262207,8303900.0,2.0,11.639505,53.440010,59.891878,12.580000,4.030024
15998,2019-12-18,INTP.JK,20000.000000,20075.000000,19800.000000,17098.693359,2290000.0,2.0,-13.626100,49.586669,-17.159829,12.580000,4.030024


In [174]:
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2050.000000,2300.000000,2030.000000,1792.011108,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,INDF.JK,980.000000,980.000000,920.000000,638.533020,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,INTP.JK,4600.000000,5050.000000,4600.000000,3106.131348,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,UNTR.JK,4326.423828,5071.530273,4326.423828,3122.124756,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,INCO.JK,2300.000000,2325.000000,2100.000000,1733.576172,21013000.0,1.0,-1.311040,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNTR.JK,22000.000000,22375.000000,21750.000000,19864.078125,4607100.0,1.0,-78.720781,52.760272,20.354918,12.290000,0.951751
15996,2019-12-18,INCO.JK,3600.000000,3600.000000,3440.000000,3418.002930,14110300.0,2.0,27.673484,51.602337,90.205381,12.580000,4.030024
15997,2019-12-18,INDF.JK,7975.000000,7975.000000,7900.000000,7000.262207,8303900.0,2.0,11.639505,53.440010,59.891878,12.580000,4.030024
15998,2019-12-18,INTP.JK,20000.000000,20075.000000,19800.000000,17098.693359,2290000.0,2.0,-13.626100,49.586669,-17.159829,12.580000,4.030024


In [175]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2050.000000,2300.000000,2030.000000,1792.011108,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.0
1,2009-01-05,INDF.JK,980.000000,980.000000,920.000000,638.533020,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.0
2,2009-01-05,INTP.JK,4600.000000,5050.000000,4600.000000,3106.131348,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.0
3,2009-01-05,UNTR.JK,4326.423828,5071.530273,4326.423828,3122.124756,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.0
4,2009-01-06,INCO.JK,2300.000000,2325.000000,2100.000000,1733.576172,21013000.0,1.0,-1.311040,0.000000,66.666667,38.560001,0.0
5,2009-01-06,INDF.JK,1000.000000,1110.000000,980.000000,664.595581,346017500.0,1.0,0.584737,100.000000,66.666667,38.560001,0.0
6,2009-01-06,INTP.JK,5100.000000,5250.000000,5050.000000,3198.392822,2973000.0,1.0,2.069969,100.000000,66.666667,38.560001,0.0
7,2009-01-06,UNTR.JK,5143.637207,5335.922852,4759.066406,3048.140625,23181674.0,1.0,-1.659900,0.000000,66.666667,38.560001,0.0
8,2009-01-07,INCO.JK,2300.000000,2650.000000,2275.000000,2025.751831,58399500.0,2.0,7.389824,83.798928,100.000000,43.389999,0.0
9,2009-01-07,INDF.JK,1020.000000,1080.000000,1020.000000,664.595581,147142500.0,2.0,0.745071,100.000000,53.392705,43.389999,0.0


In [176]:
processed_full.nunique()

date           2651
tic               4
open           1775
high           1835
low            1809
close          4831
volume         9764
day               5
macd          10601
rsi_30         9615
cci_30        10599
vix            1441
turbulence     2397
dtype: int64

In [177]:
#memperbarui dataframe dengan bentuk harga per lot

processed_full_lot = processed_full[['open','high','low','close']].mul(100)

#drop some columns
processed_full_noOHLC = processed_full.drop(['open','high','low','close'], axis=1)
  
processed_full_lot = pd.concat([processed_full_noOHLC, processed_full_lot], axis=1)

processed_full_lot = processed_full_lot[['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'vix', 'turbulence']]

processed_full_lot

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2.050000e+05,2.300000e+05,2.030000e+05,1.792011e+05,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,INDF.JK,9.800000e+04,9.800000e+04,9.200000e+04,6.385330e+04,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,INTP.JK,4.600000e+05,5.050000e+05,4.600000e+05,3.106131e+05,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,UNTR.JK,4.326424e+05,5.071530e+05,4.326424e+05,3.122125e+05,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,INCO.JK,2.300000e+05,2.325000e+05,2.100000e+05,1.733576e+05,21013000.0,1.0,-1.311040,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNTR.JK,2.200000e+06,2.237500e+06,2.175000e+06,1.986408e+06,4607100.0,1.0,-78.720781,52.760272,20.354918,12.290000,0.951751
15996,2019-12-18,INCO.JK,3.600000e+05,3.600000e+05,3.440000e+05,3.418003e+05,14110300.0,2.0,27.673484,51.602337,90.205381,12.580000,4.030024
15997,2019-12-18,INDF.JK,7.975000e+05,7.975000e+05,7.900000e+05,7.000262e+05,8303900.0,2.0,11.639505,53.440010,59.891878,12.580000,4.030024
15998,2019-12-18,INTP.JK,2.000000e+06,2.007500e+06,1.980000e+06,1.709869e+06,2290000.0,2.0,-13.626100,49.586669,-17.159829,12.580000,4.030024


<a id='4'></a>
# Bagian 5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style
Proses *training* meliputi mengamati perubahan harga saham, mengambil tindakan dan perhitungan *reward*. Dengan berinteraksi dengan lingkungan pasar, agen pada akhirnya akan memperoleh strategi perdagangan yang dapat memaksimalkan imbalan.

*Environment* pasar dibangun  menggunakan OpenAI Gym yang mensimulasikan pasar saham dengan data historis pasar.

<a id='4.1'></a>
## 5.1. Memisahkan Data
Data dibagi menjadi dua bagian, yaitu bagian pelatihan dan bagian pengujian dengan keterangan sebagai berikut:

Periode data pelatihan: 2009-01-01 hingga 2017-12-31

Periode data perdagangan: 2018-01-01 hingga 2022-08-15

In [178]:
train = data_split(processed_full_lot, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full_lot, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

8628
1976


In [179]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2155,2017-12-28,UNTR.JK,3457500.0,3460000.0,3422500.0,2.839685e+06,2636700.0,3.0,297.519992,55.675876,163.784450,10.18,5.337369
2156,2017-12-29,INCO.JK,285000.0,291000.0,285000.0,2.863197e+05,9727700.0,4.0,-6.167929,51.919947,61.612511,11.04,3.007362
2156,2017-12-29,INDF.JK,770000.0,777500.0,760000.0,6.258995e+05,12848100.0,4.0,-15.327369,44.909596,53.529935,11.04,3.007362
2156,2017-12-29,INTP.JK,2150000.0,2195000.0,2150000.0,1.754518e+06,1534000.0,4.0,288.717219,58.947150,197.982423,11.04,3.007362
2156,2017-12-29,UNTR.JK,3460000.0,3540000.0,3457500.0,2.905342e+06,4163100.0,4.0,377.482726,58.036918,202.252625,11.04,3.007362


In [180]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,INCO.JK,292000.0,304000.0,292000.0,3.001898e+05,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799054
0,2018-01-02,INDF.JK,765000.0,770000.0,755000.0,6.197431e+05,6875700.0,1.0,-14.553045,43.053753,30.342646,9.77,13.799054
0,2018-01-02,INTP.JK,2195000.0,2300000.0,2135000.0,1.838447e+06,1017900.0,1.0,471.242271,62.624347,199.526840,9.77,13.799054
0,2018-01-02,UNTR.JK,3540000.0,3540000.0,3410000.0,2.802752e+06,3701100.0,1.0,394.738224,53.290352,114.551111,9.77,13.799054
1,2018-01-03,INCO.JK,305000.0,311000.0,304000.0,3.021713e+05,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300812


In [181]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 21


In [182]:
buy_cost_list = [0.0023] * stock_dimension
sell_cost_list = [0.0054] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

<a id='4.2'></a>
## 5.2. Lingkungan untuk Pelatihan *(Training)*



In [183]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Bagian 6: Melatih Agen DRL
* Algoritma DRL dibangun dengan menggunakan **Stable Baselines 3**. 

* FinRL mencakup algoritma DRL standar yang disempurnakan, seperti DQN, DDPG, DDPG Multi-Agen, PPO, SAC, A2C, dan TD3. FinRL juga mengizinkan pengguna untuk merancang algoritma DRL sendiri dengan mengadaptasi algoritma DRL yang telah disebutkan.

In [184]:
agent = DRLAgent(env = env_train)

**Agent Training: 5 algorithms (A2C, PPO, TD3, SAC, DDPG)**

<a id='5.1'></a>
##6.1. Agen 1: A2C


In [185]:
A2C_PARAMS = {
    "n_steps": 150, 
    "ent_coef": 0.0001, 
    "learning_rate": 0.0014,
}
model_a2c = agent.get_model("a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 150, 'ent_coef': 0.0001, 'learning_rate': 0.0014}
Using cuda device


In [186]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

-------------------------------------
| time/                 |           |
|    fps                | 377       |
|    iterations         | 100       |
|    time_elapsed       | 39        |
|    total_timesteps    | 15000     |
| train/                |           |
|    entropy_loss       | -5.67     |
|    explained_variance | -0.000104 |
|    learning_rate      | 0.0014    |
|    n_updates          | 99        |
|    policy_loss        | -208      |
|    reward             | 76.74853  |
|    std                | 1         |
|    value_loss         | 9.48e+05  |
-------------------------------------
day: 2156, episode: 10
begin_total_asset: 100000000.00
end_total_asset: 120458494.82
total_reward: 20458494.82
total_cost: 1636638.52
total_trades: 4389
Sharpe: 0.468
---------------------------------------
| time/                 |             |
|    fps                | 377         |
|    iterations         | 200         |
|    time_elapsed       | 79          |
|    total_timesteps    |

<a id='5.2'></a>
##6.2. Agen 2: PPO

In [187]:
PPO_PARAMS = {
    "n_steps": 50000,
    "ent_coef": 0.7,
    "learning_rate": 0.0016,
    "batch_size": 1750,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 50000, 'ent_coef': 0.7, 'learning_rate': 0.0016, 'batch_size': 1750}
Using cuda device


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:147: UserWarning: You have specified a mini-batch size of 1750, but because the `RolloutBuffer` is of size `n_steps * n_envs = 50000`, after every 28 untruncated mini-batches, there will be a truncated mini-batch of size 1000
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=50000 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [188]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

day: 2156, episode: 30
begin_total_asset: 100000000.00
end_total_asset: 597640.55
total_reward: -99402359.45
total_cost: 59350365.08
total_trades: 5702
Sharpe: -0.791
day: 2156, episode: 40
begin_total_asset: 100000000.00
end_total_asset: 259343.83
total_reward: -99740656.17
total_cost: 30598786.50
total_trades: 5428
Sharpe: -1.056
---------------------------------
| time/              |          |
|    fps             | 380      |
|    iterations      | 1        |
|    time_elapsed    | 131      |
|    total_timesteps | 50000    |
| train/             |          |
|    reward          | 30.13061 |
---------------------------------


<a id='5.3'></a>
##6.3. Agen 3: TD3

In [189]:
TD3_PARAMS = {
    "batch_size": 7000, 
    "buffer_size": 10000, 
    "learning_rate": 0.9  
}
model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 7000, 'buffer_size': 10000, 'learning_rate': 0.9}
Using cuda device


In [190]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000)

day: 2156, episode: 50
begin_total_asset: 100000000.00
end_total_asset: 247891593.53
total_reward: 147891593.53
total_cost: 8867230.37
total_trades: 6466
Sharpe: 0.481
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 98        |
|    time_elapsed    | 87        |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 8.21e+06  |
|    critic_loss     | 5.7e+16   |
|    learning_rate   | 0.9       |
|    n_updates       | 6471      |
|    reward          | 191.83711 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 86        |
|    time_elapsed    | 200       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 6.55e+06  |
|    critic_loss     | 7.09e+12  |
|    learning_rate   | 0.9       |
|    n_updates       | 1509

<a id='5.4'></a>
##6.4. Agen 4: SAC

In [191]:
SAC_PARAMS = {
    "ent_coef": 1.25,
    "learning_rate": 0.0001,
    "batch_size": 4000,
    "buffer_size": 5000,
    "learning_starts": 14,
    
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'ent_coef': 1.25, 'learning_rate': 0.0001, 'batch_size': 4000, 'buffer_size': 5000, 'learning_starts': 14}
Using cuda device


In [192]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 64        |
|    time_elapsed    | 132       |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | -8.26e+03 |
|    critic_loss     | 2.02e+05  |
|    ent_coef        | 1.25      |
|    learning_rate   | 0.0001    |
|    n_updates       | 8613      |
|    reward          | 222.59906 |
----------------------------------
day: 2156, episode: 80
begin_total_asset: 100000000.00
end_total_asset: 100322749.20
total_reward: 322749.20
total_cost: 248599.56
total_trades: 6468
Sharpe: 0.470
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 66        |
|    time_elapsed    | 260       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 1.28e+03  |
|    critic_loss     | 3.27e+04  |
|    ent_coef        | 1.25    

<a id='5.5'></a>
##6.5. Agen 5: DDPG

In [193]:
DDPG_PARAMS = {
    "batch_size": 1250, 
    "buffer_size": 50000, 
    "learning_rate": 0.0003
}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 1250, 'buffer_size': 50000, 'learning_rate': 0.0003}
Using cuda device


In [194]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=60000)

day: 2156, episode: 90
begin_total_asset: 100000000.00
end_total_asset: 107572283.40
total_reward: 7572283.40
total_cost: 229300.98
total_trades: 4312
Sharpe: 0.476
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 129       |
|    time_elapsed    | 66        |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 1.08e+04  |
|    critic_loss     | 1.56e+07  |
|    learning_rate   | 0.0003    |
|    n_updates       | 6471      |
|    reward          | 242.93318 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 115       |
|    time_elapsed    | 149       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 8.45e+03  |
|    critic_loss     | 1.97e+05  |
|    learning_rate   | 0.0003    |
|    n_updates       | 15099  

<a id='6'></a>
# Bagian 7: Melakukan Jual-Beli

<a id='6.1'></a>
## 7.1. Performa di Dalam Sampel

Asumsikan modal awal adalah Rp100.000.000.

In [195]:
data_risk_indicator = processed_full_lot[(processed_full_lot.date<TRAIN_END_DATE) & (processed_full_lot.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [196]:
insample_risk_indicator.vix.describe()

count    2157.000000
mean       18.666583
std         7.813415
min         9.140000
25%        13.370000
50%        16.299999
75%        21.540001
max        56.650002
Name: vix, dtype: float64

In [197]:
insample_risk_indicator.vix.quantile(0.996)

48.84047967529307

In [198]:
insample_risk_indicator.turbulence.describe()

count    2157.000000
mean        3.740825
std         5.794686
min         0.000000
25%         0.684352
50%         2.070514
75%         4.635442
max        86.425129
Name: turbulence, dtype: float64

In [199]:
insample_risk_indicator.turbulence.quantile(0.996)

39.048710611468394

<a id='6.2'></a>
##7.2. Performa di Luar Sampel

FinRL dapat diatur dengan cara melakukan pelatihan ulang secara berkala, misalnya, pelatihan ulang setiap tiga bulan, bulanan, atau mingguan. 

Pada notebook ini, *hyperparameter* hanya diatur satu kali dengan menggunakan data sampel selama periode yang telah ditetapkan. Sehingga, terjadi beberapa peluruhan alfa seiring dengan perpanjangan tanggal perdagangan.

Banyak *hyperparameter* – mis. learning rate, mempengaruhi proses pembelajaran dan biasanya ditentukan dengan menguji beberapa variasi.

In [200]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

<a id='6.3'></a>
##7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi

<a id='6.3.1'></a>
###7.3.1. Agen 1: A2C

In [201]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym)

hit end!


<a id='6.3.2'></a>
###7.3.2. Agen 2: PPO

In [202]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym)

hit end!


<a id='6.3.3'></a>
###7.3.3. Agen 3: TD3

In [203]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym)

hit end!


<a id='6.3.4'></a>
###7.3.4. Agen 4: SAC

In [204]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


<a id='6.3.5'></a>
###7.3.5. Agen 5: DDPG

In [205]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

hit end!


<a id='7'></a>
# Bagian 8: Performa Backtesting
Backtesting memiliki peran kunci dalam mengevaluasi kinerja strategi perdagangan. Alat backtesting otomatis lebih disukai karena dapat meminimalisir kesalahan manusia. Backtesting dapat dilakukan dengan menggunakan paket Quantopian pyfolio untuk menguji strategi perdagangan pada notebook ini. Backtesting tersebut mudah digunakan dan terdiri dari berbagai plot yang memberikan gambaran komprehensif tentang kinerja strategi perdagangan.

<a id='7.1'></a>
## 8.1 Status BackTesting


###8.1.1 Agen A2C

In [206]:
print("===========Get Backtest Results of A2C Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_a2c)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_a2c_"+now+'.csv')

===========Get Backtest Results of A2C Model===========
Annual return          0.060111
Cumulative returns     0.121236
Annual volatility      0.281533
Sharpe ratio           0.348013
Calmar ratio           0.203311
Stability              0.202448
Max drawdown          -0.295662
Omega ratio            1.062851
Sortino ratio          0.512477
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.133085
Daily value at risk   -0.035081
dtype: float64


###8.1.2 Agen ppo

In [207]:
print("===========Get Backtest Results of PPO Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ppo_"+now+'.csv')

===========Get Backtest Results of PPO Model===========
Annual return          0.001048
Cumulative returns     0.002055
Annual volatility      0.297184
Sharpe ratio           0.151386
Calmar ratio           0.003461
Stability              0.296010
Max drawdown          -0.302713
Omega ratio            1.025801
Sortino ratio          0.223219
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.093470
Daily value at risk   -0.037263
dtype: float64


###8.1.3 Agen TD3

In [208]:
print("===========Get Backtest Results of TD3 Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_td3)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_td3_"+now+'.csv')

===========Get Backtest Results of TD3 Model===========
Annual return         -0.037250
Cumulative returns    -0.071715
Annual volatility      0.418317
Sharpe ratio           0.116605
Calmar ratio          -0.088330
Stability              0.262397
Max drawdown          -0.421715
Omega ratio            1.020412
Sortino ratio          0.174546
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.082528
Daily value at risk   -0.052509
dtype: float64


###8.1.4 Agen SAC

In [209]:
print("===========Get Backtest Results of SAC Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_sac)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_sac_"+now+'.csv')

===========Get Backtest Results of SAC Model===========
Annual return          0.060800
Cumulative returns     0.122664
Annual volatility      0.295850
Sharpe ratio           0.347770
Calmar ratio           0.207905
Stability              0.023751
Max drawdown          -0.292440
Omega ratio            1.062388
Sortino ratio          0.502711
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.072977
Daily value at risk   -0.036865
dtype: float64


###8.1.5 Agen DDPG

In [210]:
print("===========Get Backtest Results of DDPG Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ddpg)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ddpg_"+now+'.csv')

===========Get Backtest Results of DDPG Model===========
Annual return         -0.173171
Cumulative returns    -0.311175
Annual volatility      0.344538
Sharpe ratio          -0.380780
Calmar ratio          -0.365391
Stability              0.836461
Max drawdown          -0.473932
Omega ratio            0.936792
Sortino ratio         -0.532684
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.069286
Daily value at risk   -0.043928
dtype: float64


###8.1.6. Baseline Stats

In [211]:
#baseline stats
print("===========Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^JKII", 
        start = df_account_value_a2c.loc[0,'date'],
        end = df_account_value_a2c.loc[len(df_account_value_a2c)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

===========Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (478, 8)
Annual return         -0.041972
Cumulative returns    -0.078113
Annual volatility      0.185511
Sharpe ratio          -0.138819
Calmar ratio          -0.175216
Stability              0.093175
Max drawdown          -0.239543
Omega ratio            0.977818
Sortino ratio         -0.189296
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.900451
Daily value at risk   -0.023474
dtype: float64
